In [16]:
# -*- coding: utf-8 -*-
#!/usr/bin/python

import RPi.GPIO as GPIO
import os
import numpy as np
import smbus
import time
import pickle

from bokeh.plotting import figure
from bokeh.io import output_file, show, output_notebook
output_notebook()

class ADXL345():
    def __init__(self):
        
        self.DevAdr = 0x53
        myBus = ""
        if GPIO.RPI_INFO['P1_REVISION'] == 1:
            myBus = 0
        else:
            myBus = 1
            
        self.b = smbus.SMBus(myBus)
        
        self.b.write_byte_data(self.DevAdr, 0x2C, 0x0B) # BandwidthRate　originally 0B 
        self.b.write_byte_data(self.DevAdr, 0x31, 0x00) # DATA_FORMAT 10bit 2g
        self.b.write_byte_data(self.DevAdr, 0x38, 0x00) # FIFO_CTL OFF
        self.b.write_byte_data(self.DevAdr, 0x2D, 0x08) # POWER_CTL Enable
    
    def getOffset(self):
        data =[]
        for a in range(NO_OF_OFFSET_MEASURE):
            input = self.getData()
            data.append(input)
            time.sleep(SAMPLING_CYCLE)
        data = np.array(data)
        offset = data.mean(axis = 0)
        
        return offset

    def getData(self):
        output = []
        data = self.b.read_i2c_block_data(self.DevAdr, 0x32, 6)
        for i in range(0,6,2):
            sign = data[i + 1] & 0x80
            tmp = data[i + 1] & 0x7F
            tmp = tmp << 8
            tmp = tmp | data[i]

            if sign > 0:
                output.append(tmp - 32768)
            else:
                output.append(tmp)

        return np.array(output)

def main():
    myADXL345 = ADXL345()

    offset = myADXL345.getOffset()

    Data =[]
    print "Start"
    
    ema = np.zeros(3)
    for a in range(NO_OF_SAMPLES):
        data = myADXL345.getData()
        data = np.array(data - offset)
        ema = 0.9 * ema + 0.1 * data
        Data.append(ema)
        time.sleep(SAMPLING_CYCLE)
        
    print "Stop"
    
    # dump data
    with open('Data.pkl', 'wb') as f:
        pickle.dump(Data, f)
 
    # plot figure 
    Data = list(map(list, zip(*Data)))  #Swap rows and columns
    
    t=[float(n) for n in range(0,NO_OF_SAMPLES)]
    
    pp = figure(tools='xwheel_zoom,xpan',
    title="",
    x_axis_label='t[s]',
    y_axis_label='Vibration')
    pp.line(t, Data[0],legend="Vibration", line_width=1, line_color = "blue")
    pp.line(t, Data[1],legend="Vibration", line_width=1, line_color = "red")
    pp.line(t, Data[2],legend="Vibration", line_width=1, line_color = "green")
    output_file("robot.html")
    show(pp)
    
if __name__ == "__main__":
    
    SAMPLING_CYCLE = 0.01 # in seconds
    MONITOR_TIME = 5 # in seconds
    
    NO_OF_SAMPLES = int(MONITOR_TIME * ( 1.0  / SAMPLING_CYCLE ))
    
    OFFSET_MEASURE_TIME = 0.5
    NO_OF_OFFSET_MEASURE = int(OFFSET_MEASURE_TIME * ( 1.0  / SAMPLING_CYCLE ))
    
    main()
    

Loading BokehJS ...

Start
Stop
